# Cross validation & Grid Search

In [1]:
import pandas as pd

In [2]:
wine = pd.read_csv('./wine.csv', encoding='utf-8')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
wine.columns

Index(['alcohol', 'sugar', 'pH', 'class'], dtype='object')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].values

In [5]:
target = wine[['class']].values

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_X, test_X, train_y, test_y = train_test_split(
    data,
    target,
    test_size = 0.2,
    random_state=42
)

In [8]:
sub_X, val_X, sub_y, val_y = train_test_split(
    train_X,
    train_y,
    test_size = 0.2,
    random_state = 42
)

In [9]:
sub_X.shape, sub_y.shape

((4157, 3), (4157, 1))

In [10]:
val_X.shape, val_y.shape

((1040, 3), (1040, 1))

## 1) 의사결정나무

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
dt = DecisionTreeClassifier(random_state=42)

In [13]:
dt.fit(sub_X, sub_y)

DecisionTreeClassifier(random_state=42)

In [14]:
print(dt.score(sub_X, sub_y))
print(dt.score(val_X, val_y))

0.9971133028626413
0.864423076923077


## 2) 교차 검증

In [15]:
from sklearn.model_selection import cross_validate

In [16]:
scores = cross_validate(dt, train_X, train_y)
scores

{'fit_time': array([0.0080049 , 0.00598812, 0.009974  , 0.00897765, 0.00797677]),
 'score_time': array([0.        , 0.0009973 , 0.00099564, 0.00099564, 0.00100303]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [17]:
import numpy as np

In [18]:
np.mean(scores['test_score'])

0.855300214703487

In [19]:
from sklearn.model_selection import StratifiedKFold

In [20]:
scores = cross_validate(dt, train_X, train_y, cv=StratifiedKFold())

In [21]:
np.mean(scores['test_score'])

0.855300214703487

In [22]:
splitter=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [23]:
scores=cross_validate(dt, train_X, train_y, cv=splitter)

In [24]:
np.mean(scores['test_score'])

0.8574181117533719

## 3) 하이퍼파라미터 튜닝

In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [27]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [28]:
gs.fit(train_X, train_y)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [29]:
dt2 = gs.best_estimator_

In [30]:
score1 = dt2.score(train_X, train_y)
score1

0.9615162593804117

In [31]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [32]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [33]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])

In [34]:
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

In [35]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
          'max_depth':range(5, 20, 1),
          'min_samples_split':range(2, 100, 10)
         }

In [36]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [37]:
gs.fit(train_X, train_y)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [38]:
gs.best_params_ # 최적의 파라미터 값

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [39]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

## 4) 랜덤 서치

In [40]:
from scipy.stats import uniform, randint

In [41]:
rgen = randint(0, 10)

In [42]:
rgen.rvs(10)

array([4, 2, 0, 4, 6, 4, 1, 1, 1, 1])

In [43]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102,  93,  88,  92, 124,  94, 113,  78,  99, 117], dtype=int64))

In [44]:
ugen = uniform(0, 1)

In [45]:
ugen.rvs(10)

array([0.99043201, 0.49175719, 0.67397173, 0.40162304, 0.43726774,
       0.17911879, 0.16873309, 0.7285278 , 0.26993431, 0.83183041])

In [46]:
params = {'min_impurity_decrease':uniform(0.0001, 0.001),
          'max_depth':randint(20, 50),
          'min_samples_split':randint(2, 25),
          'min_samples_leaf':randint(1, 25)
         }

In [47]:
from sklearn.model_selection import RandomizedSearchCV

In [48]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), 
                        params,
                        n_iter=100,
                        n_jobs=-1,
                        random_state=42
)

In [49]:
gs.fit(train_X, train_y)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BC6FA84908>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BC6FA84B38>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BC6FB3EE48>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BC6FB3E0F0>},
                   random_state=42)

In [50]:
gs.best_params_ # 최적의 파라미터 값

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [51]:
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

In [52]:
dt3 = gs.best_estimator_

In [53]:
score2 = dt3.score(test_X, test_y)
score2

0.86